In [1]:
%matplotlib inline

In [2]:
from __future__ import division,print_function

#import os, json
#from glob import glob
import numpy as np
#import pandas as pd
#np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

import utils; reload(utils)
from utils import *

#from numpy.random import random, permutation
#from scipy import misc, ndimage
#from scipy.ndimage.interpolation import zoom

'''
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers import Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD, RMSprop
from keras.preprocessing import image
'''

WARNING (theano.gof.compilelock): Overriding existing lock by dead process '2058' (I am process '6127')
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


'\nimport keras\nfrom keras import backend as K\nfrom keras.utils.data_utils import get_file\nfrom keras.models import Sequential, Model\nfrom keras.layers.core import Flatten, Dense, Dropout, Lambda\nfrom keras.layers import Input\nfrom keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D\nfrom keras.optimizers import SGD, RMSprop\nfrom keras.preprocessing import image\n'

### Define path to data

In [3]:
path = '/home/ubuntu/courses/deeplearning1/nbs/'
PROJ_HOME_DIR = path + 'data/statefarm/'
DATA_HOME_DIR = PROJ_HOME_DIR + 'sample/'

results_path = DATA_HOME_DIR + 'results/'
test_path = DATA_HOME_DIR + 'test/'
valid_path = DATA_HOME_DIR + 'valid/'
train_path = DATA_HOME_DIR + 'train/'

Set up kaggle config for this competition (after agreeing to rules).

In [4]:
#kg config -g -u `username` -p `password` 
#dogs-vs-cats-redux-kernels-edition
#state-farm-distracted-driver-detection
#
#!kg config -c 'state-farm-distracted-driver-detection'
#!kg download
#!unzip -q sample_submission.csv.zip
#!unzip -q imgs.zip
#!unzip -q driver_imgs_list.csv.zip

### Data set-up

In [5]:
## Create directories
#%mkdir valid
#%mkdir results
#%mkdir -p sample/train
#%mkdir -p sample/test
#%mkdir -p sample/valid
#%mkdir -p sample/results
#%mkdir -p test/unknown

Split training and validation set such that a percentage of drivers in the validation set are not in the training set.


In [6]:
## Read names of training images from .csv file
#%cd $DATA_HOME_DIR/

#driver_imgs_data=pd.read_csv('driver_imgs_list.csv')
##train_imgs = driver_imgs_data['img'].tolist()

#by_drivers = driver_imgs_data.groupby('subject')
#unique_drivers = np.random.permutation(by_drivers.groups.keys())
## Set validation set percentage with regards to training set
#val_pct = 0.2

##random.shuffle(unique_drivers)

## These are the drivers we will be entirely moving to the validation set
#val_drivers = unique_drivers[:int(len(unique_drivers) * val_pct)]

#val_data = driver_imgs_data.loc[driver_imgs_data['subject'].isin(val_drivers)]

In [7]:
## Create sub-directories in valid/
#for index, row in val_data.iterrows():
#    directory = DATA_HOME_DIR+'valid/'+row['classname']
#    if not os.path.exists(directory):
#        os.mkdir(directory)
        
## Move validation data from training data
#for index, row in val_data.iterrows():
#    os.rename(DATA_HOME_DIR+'train/'+row['classname']+'/'+row['img'],\
#              DATA_HOME_DIR+'valid/'+row['classname']+'/'+row['img'])

In [8]:
#from shutil import copyfile

#%cd $DATA_HOME_DIR/train
#%mkdir ../sample
#%mkdir ../sample/train
#%mkdir ../sample/valid
#for d in glob('c?'):
#    os.mkdir('../sample/train/'+d)
#    os.mkdir('../sample/valid/'+d)
#g = glob('c?/*.jpg')
#shuf = np.random.permutation(g)
#for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

#%cd ../valid
#g = glob('c?/*.jpg')
#shuf = np.random.permutation(g)
#for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

### Create batches

In [22]:
batch_size=64

# NB: They must be in subdirectories named based on their category
batches = get_batches(train_path, batch_size=batch_size)
test_batches = get_batches(test_path, batch_size=batch_size*2)
val_batches = get_batches(valid_path, batch_size=batch_size*2, shuffle=False)

Found 1500 images belonging to 10 classes.
Found 0 images belonging to 0 classes.
Found 1000 images belonging to 10 classes.


In [10]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, filenames,
    test_filename) = get_classes(DATA_HOME_DIR)

Found 1500 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.
Found 0 images belonging to 0 classes.


### Single conv layer
2 conv layers with max pooling followed by a simple dense network

In [11]:
def conv1(batches):
    model = Sequential([
            BatchNormalization(axis=1, input_shape=(3,224,224)),
            Convolution2D(32,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Convolution2D(64,3,3, activation='relu'),
            BatchNormalization(axis=1),
            MaxPooling2D((3,3)),
            Flatten(),
            Dense(200, activation='relu'),
            BatchNormalization(),
            Dense(10, activation='softmax')
        ])

    model.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit_generator(batches, batches.nb_sample, nb_epoch=2, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    model.optimizer.lr = 0.001
    model.fit_generator(batches, batches.nb_sample, nb_epoch=4, validation_data=val_batches, 
                     nb_val_samples=val_batches.nb_sample)
    return model

### Train

In [12]:
conv1(batches)

Epoch 1/2
1500/1500 [==============================] - 42s - loss: 1.5490 - acc: 0.5640 - val_loss: 2.3270 - val_acc: 0.1860
Epoch 2/2
1500/1500 [==============================] - 28s - loss: 0.2905 - acc: 0.9687 - val_loss: 2.1561 - val_acc: 0.1540
Epoch 1/4
1500/1500 [==============================] - 32s - loss: 0.0876 - acc: 0.9960 - val_loss: 2.1920 - val_acc: 0.1420
Epoch 2/4
1500/1500 [==============================] - 29s - loss: 0.0413 - acc: 0.9993 - val_loss: 2.2135 - val_acc: 0.2430
Epoch 3/4
1500/1500 [==============================] - 29s - loss: 0.0235 - acc: 1.0000 - val_loss: 2.2049 - val_acc: 0.2720
Epoch 4/4
1500/1500 [==============================] - 30s - loss: 0.0150 - acc: 1.0000 - val_loss: 2.1994 - val_acc: 0.3100


Width shift: move the image left and right

In [ ]:
gen_t = image.ImageDataGenerator(width_shift_range=0.1)
batches = get_batches(train_path, gen_t, batch_size=batch_size)

model = conv1(batches)

Height shift: move the image up and down

In [ ]:
gen_t = image.ImageDataGenerator(height_shift_range=0.05)
batches = get_batches(train_path, gen_t, batch_size=batch_size)

model = conv1(batches)

Random shear angles (max in radians)

In [ ]:
gen_t = image.ImageDataGenerator(shear_range=0.1)
batches = get_batches(train_path, gen_t, batch_size=batch_size)

model = conv1(batches)

Rotation: max in degrees

In [ ]:
gen_t = image.ImageDataGenerator(rotation_range=15)
batches = get_batches(train_path, gen_t, batch_size=batch_size)

model = conv1(batches)

Channel shift: randomly changing the R,G,B colors

In [ ]:
gen_t = image.ImageDataGenerator(channel_shift_range=20)
batches = get_batches(train_path, gen_t, batch_size=batch_size)

model = conv1(batches)

Found 1500 images belonging to 10 classes.
Epoch 1/2
1500/1500 [==============================] - 33s - loss: 2.5578 - acc: 0.2053 - val_loss: 3.9461 - val_acc: 0.1010
Epoch 2/2
1500/1500 [==============================] - 29s - loss: 1.8323 - acc: 0.3880 - val_loss: 2.1643 - val_acc: 0.2000
Epoch 1/4
1500/1500 [==============================] - 36s - loss: 1.4901 - acc: 0.5120 - val_loss: 2.0587 - val_acc: 0.3060
Epoch 2/4
1500/1500 [==============================] - 30s - loss: 1.3900 - acc: 0.5353 - val_loss: 2.1295 - val_acc: 0.3260
Epoch 3/4
1500/1500 [==============================] - 31s - loss: 1.2481 - acc: 0.5960 - val_loss: 2.2162 - val_acc: 0.3550
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 1.1256 - acc: 0.6340 - val_loss: 2.1721 - val_acc: 0.3390


All image augmentations at once

In [13]:
gen_t = image.ImageDataGenerator(rotation_range=15, height_shift_range=0.05, 
                shear_range=0.1, channel_shift_range=20, width_shift_range=0.1)
batches = get_batches(train_path, gen_t, batch_size=batch_size)

model = conv1(batches)

Found 1500 images belonging to 10 classes.
Epoch 1/2
1500/1500 [==============================] - 33s - loss: 2.5578 - acc: 0.2053 - val_loss: 3.9461 - val_acc: 0.1010
Epoch 2/2
1500/1500 [==============================] - 29s - loss: 1.8323 - acc: 0.3880 - val_loss: 2.1643 - val_acc: 0.2000
Epoch 1/4
1500/1500 [==============================] - 36s - loss: 1.4901 - acc: 0.5120 - val_loss: 2.0587 - val_acc: 0.3060
Epoch 2/4
1500/1500 [==============================] - 30s - loss: 1.3900 - acc: 0.5353 - val_loss: 2.1295 - val_acc: 0.3260
Epoch 3/4
1500/1500 [==============================] - 31s - loss: 1.2481 - acc: 0.5960 - val_loss: 2.2162 - val_acc: 0.3550
Epoch 4/4
1500/1500 [==============================] - 26s - loss: 1.1256 - acc: 0.6340 - val_loss: 2.1721 - val_acc: 0.3390


Anneal learning rate

In [14]:
model.optimizer.lr = 0.0001
model.fit_generator(batches, batches.nb_sample, nb_epoch=30, validation_data=val_batches, 
                 nb_val_samples=val_batches.nb_sample)

Epoch 1/30
1500/1500 [==============================] - 33s - loss: 1.0353 - acc: 0.6553 - val_loss: 2.2600 - val_acc: 0.3000
Epoch 2/30
1500/1500 [==============================] - 27s - loss: 1.0145 - acc: 0.6687 - val_loss: 2.2322 - val_acc: 0.3060
Epoch 3/30
1500/1500 [==============================] - 31s - loss: 0.9264 - acc: 0.7153 - val_loss: 2.0182 - val_acc: 0.3710
Epoch 4/30
1500/1500 [==============================] - 30s - loss: 0.8312 - acc: 0.7367 - val_loss: 2.0489 - val_acc: 0.3830
Epoch 5/30
1500/1500 [==============================] - 27s - loss: 0.8331 - acc: 0.7447 - val_loss: 2.0979 - val_acc: 0.3810
Epoch 6/30
1500/1500 [==============================] - 34s - loss: 0.7298 - acc: 0.7667 - val_loss: 1.8315 - val_acc: 0.4740
Epoch 7/30
1500/1500 [==============================] - 28s - loss: 0.7515 - acc: 0.7707 - val_loss: 1.7906 - val_acc: 0.4880
Epoch 8/30
1500/1500 [==============================] - 32s - loss: 0.6838 - acc: 0.7907 - val_loss: 1.7497 - val_acc:

In [25]:
model.save_weights(PROJ_HOME_DIR+'sf_model.h5')

In [26]:
model.load_weights(PROJ_HOME_DIR+'sf_model.h5')

In [24]:
model_preds = model.predict_generator(test_batches, test_batches.nb_sample)

Exception in thread Thread-53:
Traceback (most recent call last):
  File "/home/ubuntu/anaconda2/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/home/ubuntu/anaconda2/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/engine/training.py", line 425, in data_generator_task
    generator_output = next(generator)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py", line 593, in next
    index_array, current_index, current_batch_size = next(self.index_generator)
  File "/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/preprocessing/image.py", line 441, in _flow_index
    current_index = (self.batch_index * batch_size) % N
ZeroDivisionError: integer division or modulo by zero



### Create .csv file for submission to Kaggle
You must submit a csv file with the image file name, and a probability for each class.
The order of the rows does not matter. The file must have a header and should look like the following:

    img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
    img_0.jpg,1,0,0,0,0,...,0
    img_1.jpg,0.3,0.1,0.6,0,...,0
    ...

In [24]:
input_ids = [int(x[len('unknown/'):-len('.jpg')]) for x in batches.filenames]
    
list_out = sorted(zip(input_ids,np.clip(preds[:,1],0.05,0.95)))

list_out = [('id','label')]+list_out # add column labels

print(list_out[:15])

[('id', 'label'), (1, 0.94999999), (2, 0.94999999), (3, 0.94999999), (4, 0.94999999), (5, 0.050000001), (6, 0.050000001), (7, 0.050000001), (8, 0.050000001), (9, 0.050000001), (10, 0.050000001), (11, 0.050000001), (12, 0.94999999), (13, 0.050000001), (14, 0.050000001)]


In [25]:
import csv

with open("data/dogscats/kaggle/dogscats_predictions4.csv", "wb") as f:
    writer = csv.writer(f)
    writer.writerows(list_out)

In [26]:
! kg submit data/dogscats/kaggle/dogscats_predictions4.csv

Starting new HTTPS connection (1): www.kaggle.com

